In [2]:
import pandas as pd
from numpy import *
from sklearn.ensemble  import  RandomForestRegressor
from sklearn.model_selection  import  train_test_split
import warnings
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import os
import sys

## need to change working directory when using Spyder 
path_current = os.getcwd()
path_root    = os.path.dirname(path_current)
print(path_root)

g:\Shared drives\gdrive\_paper\_phd_dissertation\ch4_dead_zone\Dead_Zone_telecoupling


In [7]:
index_list = ['Depth','water_temp_surface','water_temp_dif',
              'chlor_a','nflh','Rrs_678',
              'poc', 'sst', 'Rrs_667', 'Rrs_645',            ## not included by Zilong
              'Rrs_443', 'Rrs_412', 'Rrs_469', 'Rrs_488',    ## not included by Zilong
              'Rrs_555', 'Rrs_547', 'Rrs_531',               ## not included by Zilong, lowerest importance
              'wind_speed',         # not included
              'doy_img', 
              'nday_before',        # not included
              'velocityu_surface',  # not included
              'velocityu_dif', 
              'salinity_surface',
              'salinity_dif',
              'lon','lat']


# path_data = 'D:\\D\\' ## Zilong
path_data = path_root + '\\data\\from_gee\\rs_do_sample\\rs_do_sample_lagByDay\\'


## to fit the model by year, or use all data as a whole
# by_year  = 'yes'
by_year  = 'no'


## create empty list in order to collect results from each run
lag_list = [];
yr_list  = [];

random_i = []
mae_list = []
mse_list = []
r2_list = []

zong_r2_list  = []
zong_mae_list = []
zong_mse_list = []

 
importance_df = pd.DataFrame()
for lag in range(0,81): # (0, 81) for initial test; (31,32) for the best lag
        print('lag: ' + str(lag))
        samples = pd.read_excel(path_data + 'RS_do_bottom_sample_2000_2019_byday_' + str(lag) + 'dayBefore.xlsx');

        for ite in index_list:  
            # 删除空值
            samples = samples[samples[ite].notna()]
        obj_num = []
        
        for yy in range(2003,2004): ## if by_year  = 'no', this line won't be used. 
            
            if by_year == 'yes':
                yr = yy
            else:
                yr = 1
  
            ## data test finds that only using data from 'SEAMAP' can achieve better accuracy
            # samples2 = samples[samples['Source'] == 'SEAMAP'];
            samples2 = samples;
            
            X = samples2[index_list]
            Y = samples2[['DO']]
            # mae_list = []
            # mse_list = []
            # r2_list = []
            for i in range(20):   #循环20次验证精度
                X_train ,  X_test ,  y_train ,  y_test  =  train_test_split( X , Y ,  test_size  = 0.3)
                regr = RandomForestRegressor() #参数默认
                regr.fit(X_train, y_train.values.ravel())
                warnings.filterwarnings('ignore')
                predictions = regr.predict(X_test)
                result = X_test
                result['price'] = y_test
                result['prediction'] = predictions.tolist()
                ## save the result
                if lag in [31, 32]:
                    file_name = path_root + '\\data\\results_RF\\rf_prediction_lag_' + str(lag) + 'loop' + str(i) + '.csv'
                    result.to_csv(file_name, index=False)
                
                mae = mean_absolute_error(y_test.values.ravel(), predictions)
                mse = mean_squared_error(y_test.values.ravel(), predictions)
                r2 = r2_score(y_test.values.ravel(), predictions)
                # print(r2)
                mse_list.append(mse)
                mae_list.append(mae)
                r2_list.append(r2)
                lag_list.append(lag)
                yr_list.append(yr)
                random_i.append(i)
                
                ## to get the variable importance score
                import_list = []
                score_list  = []
                
                characteristics = X.columns
                importances = list(regr.feature_importances_)
                characteristics_importances = [(characteristic, round(importance, 2)) for characteristic, importance in zip(characteristics, importances)]
                characteristics_importances = sorted(characteristics_importances, key = lambda x: x[1], reverse = True)
                for pair in characteristics_importances:
                    import_list.append(pair[0])
                    score_list.append(pair[1])
                # [print('Variable: {:20} Importance: {}'.format(*pair)) for pair in characteristics_importances]; #打印变量重要性
                imp_dict = {'year': yr,
                            'lag': lag,
                            'random_i': i,
                            'var': import_list,
                            'score': score_list}
                imp_df = pd.DataFrame(imp_dict)

                ##  Starting with pandas version 1.4.0, 
                ##  the append method is deprecated and will be removed in a future release. 
                ##  Instead, you should use the concat() function to combine DataFrames
                # importance_df = importance_df.append(imp_df, ignore_index=True) 
                importance_df = pd.concat([importance_df, imp_df], ignore_index=True)


            ## calculate the mean of 20 models
            # print('Mean Absolute Error:',round(mean(mae_list),2))
            # print('Mean Squared Error:',round(mean(mse_list),2))
            print('R-squared scores:',round(mean(r2_list),4))
            # zong_r2_list.append(round(mean(r2_list),4))
            # zong_mae_list.append(round(mean(mae_list), 4))
            # zong_mse_list.append(round(mean(mse_list), 4))
            
            


lag: 0
R-squared scores: 0.5729
lag: 1
R-squared scores: 0.5778
lag: 2
R-squared scores: 0.585
lag: 3
R-squared scores: 0.5866
lag: 4
R-squared scores: 0.5898
lag: 5
R-squared scores: 0.5907
lag: 6
R-squared scores: 0.5895
lag: 7
R-squared scores: 0.5919
lag: 8
R-squared scores: 0.5882
lag: 9
R-squared scores: 0.5865
lag: 10
R-squared scores: 0.5872
lag: 11
R-squared scores: 0.5885
lag: 12
R-squared scores: 0.5875
lag: 13
R-squared scores: 0.585
lag: 14
R-squared scores: 0.5832
lag: 15
R-squared scores: 0.5842
lag: 16
R-squared scores: 0.583
lag: 17
R-squared scores: 0.5825
lag: 18
R-squared scores: 0.5818
lag: 19
R-squared scores: 0.5826
lag: 20
R-squared scores: 0.5809
lag: 21
R-squared scores: 0.5789
lag: 22
R-squared scores: 0.5794
lag: 23
R-squared scores: 0.5813
lag: 24
R-squared scores: 0.5818
lag: 25
R-squared scores: 0.5826
lag: 26
R-squared scores: 0.5844
lag: 27
R-squared scores: 0.584
lag: 28
R-squared scores: 0.5837
lag: 29
R-squared scores: 0.5821
lag: 30
R-squared scores

In [9]:

# print(zong_r2_list)
# print(zong_mae_list)
# print(zong_mse_list)

# print(r2_list)
# print(lag_list)
# print(score_list)
# print(characteristics_importances)

# put the raw data into a dictionary of lists, and then convert it to a dataframe, then csv
dict = {'year': yr_list,
        'lag': lag_list,
        'random_i': random_i,
        'r2':  r2_list,
        'mae': mae_list,
        'mse': mse_list}
# creating a dataframe from list
df = pd.DataFrame(dict)

file_name = path_root + '\\data\\results_RF\\rf_r2_mse_mae_' + 'by_year' + by_year.upper() + '_' + str(len(index_list)) + 'vars.csv'; print(file_name)
df.to_csv(file_name, index=False)

file_name = path_root + '\\data\\results_RF\\rf_importance_' + 'by_year' + by_year.upper() + '_' + str(len(index_list)) + 'vars.csv'; print(file_name)
importance_df.to_csv(file_name, index=False)



g:\Shared drives\gdrive\_paper\_phd_dissertation\ch4_dead_zone\Dead_Zone_telecoupling\data\results_RF\rf_r2_mse_mae_by_yearNO_26vars.csv
g:\Shared drives\gdrive\_paper\_phd_dissertation\ch4_dead_zone\Dead_Zone_telecoupling\data\results_RF\rf_importance_by_yearNO_26vars.csv
